<a href="https://colab.research.google.com/github/BikramKC7/Fusemachines-Assessment/blob/main/Assignment_RL_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Representation Learning Assignment
This assignment covers key topics in representation learning using PyTorch datasets. You will implement tasks for Contrastive Learning, Energy-Based Models. Use torchvision.datasets.MNIST for Exercises 1, 2.

**Total Points: 12**
- Exercise 1: Contrastive Learning (8 points)
- Exercise 2: Energy-Based Models (4 points)

Import necessary libraries and load datasets as needed.


In [1]:
# --- Install dependencies (CPU-only build by default) ---
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cpu
!pip install -q torch_geometric

# --- Imports ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader

print("Torch:", torch.__version__, "Torchvision:", torchvision.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.9 MB/s eta 0:00:00
Torch: 2.8.0+cu126 Torchvision: 0.23.0+cu126
Using device: cuda


In [2]:
# --- MNIST (for Exercises 1, 2) ---
mnist_transform = transforms.ToTensor()

mnist_train_full = datasets.MNIST(
    root="./data", train=True, download=True, transform=mnist_transform
)
mnist_test_full = datasets.MNIST(
    root="./data", train=False, download=True, transform=mnist_transform
)

# Smaller subsets for quicker experiments
train_indices = list(range(1000))
test_indices = list(range(200))

mnist_train = Subset(mnist_train_full, train_indices)
mnist_test = Subset(mnist_test_full, test_indices)

print(f"MNIST Train Subset: {len(mnist_train)} samples")
print(f"MNIST Test Subset: {len(mnist_test)} samples")

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.11MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.28MB/s]

MNIST Train Subset: 1000 samples
MNIST Test Subset: 200 samples


## Exercise 1: Contrastive Learning (8 points)

We’ll simulate contrastive learning on MNIST images by computing similarities, pairs, and losses.

### Task 1a (1 point): Cosine Similarity

Compute row-wise cosine similarity between two batches of embeddings.

In [7]:
### Ex-1-Task-1
import torch
import torch.nn.functional as F

def compute_similarity(emb1, emb2):
    """
    emb1, emb2: (B, D)
    Returns: (B,) cosine similarities
    """
    ### BEGIN SOLUTION
    sims = F.cosine_similarity(emb1, emb2, dim=1)
    return sims
    ### END SOLUTION

# Test
x = torch.randn(5, 10)
y = x.clone()
print(compute_similarity(x, y))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [4]:
# INTENTIONALLY LEFT BLANK

### Task 1b: Positive & Negative Pairs (2 points)

Generate positive pairs (same sample) and negative pairs (different sample) for a batch.
Simulate negatives by randomly shuffling batch.


In [8]:
### Ex-1-Task-2
def generate_pairs(batch):
    """
    batch: (B, D)
    Returns:
        pos: (B, D)
        neg: (B, D)
    """
    ### BEGIN SOLUTION
    pos = batch.clone()  # same sample = positive pair
    # random permutation for negatives
    indices = torch.randperm(batch.size(0))
    neg = batch[indices]
    return pos, neg
    ### END SOLUTION

# Test
batch = torch.randn(5, 8)
pos, neg = generate_pairs(batch)
print("Positive pairs:\n", pos)
print("Negative pairs:\n", neg)

Positive pairs:
 tensor([[ 1.5942,  0.6292,  1.0779,  1.5294, -1.3172, -0.9009,  0.6257,  0.1390],
        [-0.3036, -0.7643,  0.5212, -0.5870,  0.2196,  0.8548,  0.3195, -1.3961],
        [-0.3322,  0.0889,  0.8502, -0.0164, -1.0871, -0.3358,  0.2636,  0.0878],
        [-1.2035, -2.8497, -1.7860,  0.2662,  2.1504, -0.6107, -0.9836, -0.4419],
        [-0.2168, -1.5704, -0.9653, -0.0810,  1.0903, -0.0704, -1.8542,  0.8154]])
Negative pairs:
 tensor([[ 1.5942,  0.6292,  1.0779,  1.5294, -1.3172, -0.9009,  0.6257,  0.1390],
        [-1.2035, -2.8497, -1.7860,  0.2662,  2.1504, -0.6107, -0.9836, -0.4419],
        [-0.2168, -1.5704, -0.9653, -0.0810,  1.0903, -0.0704, -1.8542,  0.8154],
        [-0.3036, -0.7643,  0.5212, -0.5870,  0.2196,  0.8548,  0.3195, -1.3961],
        [-0.3322,  0.0889,  0.8502, -0.0164, -1.0871, -0.3358,  0.2636,  0.0878]])


In [6]:
# INTENTIONALLY LEFT BLANK

### Task 1c:NT-Xent Loss (2 points)

Implement normalized temperature-scaled cross-entropy loss for a batch.

In [24]:
### Ex-1-Task-3
import torch
import torch.nn.functional as F

def nt_xent_loss(sims, temp=0.5):
    """
    sims: (B, B) similarity matrix
    temp: scalar temperature
    Returns: scalar loss
    """
    ### BEGIN SOLUTION
    B = sims.size(0)
    # scale by temperature
    sims = sims / temp
    # numerator: diagonal entries (positive pairs)
    numerator = torch.exp(torch.diag(sims))
    # denominator: sum over row excluding self
    mask = torch.eye(B, dtype=torch.bool, device=sims.device)
    exp_sims = torch.exp(sims) * (~mask)  # zero out diagonal
    denom = exp_sims.sum(dim=1)
    # compute loss
    loss = -torch.log(numerator / (denom + 1e-8)).mean()
    return loss
    ### END SOLUTION

# Test
sims = torch.randn(3, 3)
print("NT-Xent loss:", nt_xent_loss(sims))

NT-Xent loss: tensor(2.2880)


In [10]:
# Quick visible test with identity similarity
sims = torch.eye(4)  # 4x4 identity matrix
loss = nt_xent_loss(sims)
print("NT-Xent loss:", loss.item())  # Should be > 0


NT-Xent loss: -0.9013876914978027


In [11]:
# INTENTIONALLY LEFT BLANK


### Task 1d: Augment MNIST Image (2 points)

Apply random small rotation ±10° to a single MNIST image.


In [22]:
### Ex-1-Task-4
from torchvision import transforms

augment = transforms.RandomRotation(degrees=10)

def augment_image(img):
    """
    img: (28, 28) tensor
    Returns: augmented image (28,28)
    """
    ### BEGIN SOLUTION
    # Add channel dimension
    img = img.unsqueeze(0)  # (1,28,28)
    # Apply rotation
    aug_img = augment(img)
    # Remove channel dimension to return (28,28)
    aug_img = aug_img.squeeze(0)
    return aug_img
    ### END SOLUTION

In [13]:
img, label = mnist_train[0]
aug_img = augment_image(img)
print("Original shape:", img.shape)
print("Augmented shape:", aug_img.shape)

Original shape: torch.Size([1, 28, 28])
Augmented shape: torch.Size([28, 28])


In [14]:
# INTENTIONALLY LEFT BLANK

### Task 1e: Contrastive Embedding Distance (1 point)

Compute Euclidean distance between two batches of embeddings row-wise.

In [15]:
### Ex-1-Task-5
import torch

def embedding_distance(emb1, emb2):
    """
    emb1, emb2: (B, D)
    Returns: (B,) Euclidean distances
    """
    ### BEGIN SOLUTION
    distances = torch.norm(emb1 - emb2, dim=1)
    return distances
    ### END SOLUTION

# --- Test Example ---
batch_size, dim = 4, 5
emb1 = torch.randn(batch_size, dim)
emb2 = torch.randn(batch_size, dim)

distances = embedding_distance(emb1, emb2)
print("Embeddings 1:\n", emb1)
print("Embeddings 2:\n", emb2)
print("Row-wise Euclidean distances:\n", distances)

Embeddings 1:
 tensor([[ 0.9018,  1.5010, -0.0951, -0.0545, -0.6905],
        [-0.7705,  0.2701, -0.4488,  0.0722, -2.9518],
        [ 0.9429,  1.3173, -0.6367,  0.2986, -1.3519],
        [-1.0252, -0.1940, -0.2669, -0.5251, -2.2298]])
Embeddings 2:
 tensor([[-0.2727, -0.6831,  1.3967, -0.8549, -0.5039],
        [-0.2008, -1.4676,  0.3769,  0.6906,  0.1538],
        [ 1.4579, -0.8969,  0.3466,  1.9297,  1.3521],
        [-0.3813, -0.3001, -0.7675,  0.3332, -1.5757]])
Row-wise Euclidean distances:
 tensor([3.0084, 3.7488, 4.0133, 1.3568])


In [16]:
# INTENTIONALLY LEFT BLANK

## Exercise 2: Energy-Based Models (4 points)



### Task 2a (2 points): Define Energy Function

Energy function for binary classification: E(x,y;w)=−y⋅(w⋅x)

In [17]:
### Ex-2-Task-1
def energy_function(x, y, w):
    """
    x: (784,), y: +1/-1, w: (784,)
    Returns: scalar energy
    """
    ### BEGIN SOLUTION
    energy = -y * torch.dot(w, x)
    return energy
    ### END SOLUTION

# Test
x = torch.randn(784)
w = torch.randn(784)
y = 1
print("Energy:", energy_function(x, y, w))

Energy: tensor(-7.1856)


In [18]:
# INTENTIONALLY LEFT BLANK

### Task 2b (2 points): Perceptron Loss (2 point)

Compute perceptron loss: max(0, E(x, y_true) - E(x, y_pred))

In [19]:
### Ex-2-Task-2
def perceptron_loss(x, y_true, y_pred, w):
    """
    x: (784,), y_true, y_pred: +1/-1
    w: (784,)
    Returns: scalar loss
    """
    ### BEGIN SOLUTION
    e_true = energy_function(x, y_true, w)
    e_pred = energy_function(x, y_pred, w)
    loss = torch.clamp(e_true - e_pred, min=0)
    return loss
    ### END SOLUTION

# Test
print("Perceptron loss:", perceptron_loss(x, 1, -1, w))

Perceptron loss: tensor(0.)


In [20]:
# INTENTIONALLY LEFT BLANK